In [6]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, SimpleRNN
from keras.optimizers import SGD, Adam
from keras import backend as K
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#import PIL
from PIL import Image
import pandas as pd
import random
from keras.callbacks import TensorBoard

In [84]:
data_train = pd.read_csv('Features_train_copy.csv')
data_test = pd.read_csv('Features_test_copy.csv')
labels_train = data_train['Diagnostics']
labels_test = data_test['Diagnostics']

del data_test['Diagnostics']
del data_train['Diagnostics']

data_test.head(6)

,'AGE','PTEDUCAT','PTGENDER_Male','PTETHCAT_Not Hisp/Latino','PTETHCAT_Unknown','PTRACCAT_Asian','PTRACCAT_Black','PTRACCAT_Hawaiian/Other PI','PTRACCAT_More than one','PTRACCAT_Unknown',...,'EcogSPDivatt_Mean','EcogSPDivatt_Std','EcogSPMem_MaxTime','EcogSPMem_Delta','EcogSPMem_Mean','EcogSPMem_Std','EcogSPTotal_MaxTime','EcogSPTotal_Delta','EcogSPTotal_Mean','EcogSPTotal_Std']
0,0.457,0.714,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.250,0.3830,0.000,0.552,0.333,0.197,0.000,0.529,0.2220,0.1870
1,0.666,1.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.346,0.2100,0.107,0.457,0.400,0.188,0.107,0.434,0.2940,0.1480
2,0.928,0.714,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.346,0.2100,0.107,0.457,0.400,0.188,0.107,0.434,0.2940,0.1480
3,0.457,0.286,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.346,0.2100,0.107,0.457,0.400,0.188,0.107,0.434,0.2940,0.1480
4,0.192,0.714,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.778,0.0836,0.000,0.414,0.792,0.129,0.000,0.417,0.6470,0.0635
5,0.281,0.929,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.050,0.2130,1.000,0.920,0.165,0.551,1.000,0.556,0.0829,0.1950


In [ ]:
x_train = np.array(data_train)
x_test = np.array(data_test)
y_train = np.array(labels_train)
y_test = np.array(labels_test)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_test = x_test.reshape(x_test.shape[0],1, x_test.shape[1])
x_train = x_train.reshape(x_train.shape[0],1, x_train.shape[1])

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [68]:
# create an empty network model
model = Sequential()
model.add(SimpleRNN(4, input_shape=x_train.shape[1:]))
# --- input layer ---
#model.add(Conv2D(16, kernel_size=(5, 5), activation='sigmoid', input_shape=input_shape))
# --- max pool ---
#model.add(MaxPooling2D(pool_size=(2, 2)))

# --- next layer ---
# we could double the number of filters as max pool made the 
# feature maps much smaller 
# just not doing this to improve runtime
#model.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))
# --- max pool ---
#model.add(MaxPooling2D(pool_size=(2, 2)))

# flatten for fully connected classification layer
#model.add(Flatten())
# note that the 10 is the number of classes we have
# the classes are mutually exclusive so softmax is a good choice
# --- fully connected layer ---
#model.add(Dense(64, activation='sigmoid'))
# --- classification ---
model.add(Dense(1, activation='relu'))

# prints out a summary of the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_14 (SimpleRNN)    (None, 4)                 472       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 5         
Total params: 477
Trainable params: 477
Non-trainable params: 0
_________________________________________________________________


In [69]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (actual_positives + K.epsilon())
    return recall

In [73]:
sgd = SGD(lr=0.01, momentum=0.9)
model.compile(loss="mean_squared_error",
              optimizer=sgd,
              metrics=[recall])

In [78]:
# smaller batch size means noisier gradient, but more updates per epoch
batch_size = 100
# number of iterations over the complete training data
epochs = 500

# this is now the actual training
# in addition to the training data we provide validation data
# this data is used to calculate the performance of the model over all the epochs
# this is useful to determine when training should stop
# in our case we just use it to monitor the evolution of the model over the training epochs
# if we use the validation data to determine when to stop the training or which model to save, we 
# should not use the test data, but a separate validation set. 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# once training is complete, let's see how well we have done
score = model.evaluate(x_test, y_test, verbose=0)
predictions = (model.predict(x_test)>0.5).astype(int)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1163 samples, validate on 574 samples
Epoch 1/500
1163/1163 [==============================] - 0s - loss: 0.0481 - recall: 0.9355 - val_loss: 0.0540 - val_recall: 0.9163
Epoch 2/500
1163/1163 [==============================] - 0s - loss: 0.0482 - recall: 0.9238 - val_loss: 0.0546 - val_recall: 0.9432
Epoch 3/500
1163/1163 [==============================] - 0s - loss: 0.0480 - recall: 0.9379 - val_loss: 0.0541 - val_recall: 0.9163
Epoch 4/500
1163/1163 [==============================] - 0s - loss: 0.0482 - recall: 0.9289 - val_loss: 0.0553 - val_recall: 0.9432
Epoch 5/500
1163/1163 [==============================] - 0s - loss: 0.0484 - recall: 0.9508 - val_loss: 0.0540 - val_recall: 0.9114
Epoch 6/500
1163/1163 [==============================] - 0s - loss: 0.0482 - recall: 0.9352 - val_loss: 0.0546 - val_recall: 0.9432
Epoch 7/500
1163/1163 [==============================] - 0s - loss: 0.0480 - recall: 0.9313 - val_loss: 0.0541 - val_recall: 0.9163
Epoch 8/500
1163/1163 [======

In [81]:
for i in range(len(predictions)):
    print(predictions[i][0],int(y_test[i]))

1 0
0 0
1 1
0 0
1 1
0 0
1 1
0 0
1 1
0 0
1 1
0 0
0 0
0 0
0 0
0 0
1 1
0 0
1 1
1 1
1 1
1 1
1 1
0 0
1 1
0 0
0 1
0 0
0 0
1 1
0 0
1 1
0 0
1 1
0 0
0 0
0 0
1 1
1 1
0 0
0 0
0 0
1 1
0 0
0 0
1 1
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 1
1 1
1 1
0 0
0 0
1 1
0 0
0 0
0 0
1 1
0 0
1 1
0 0
1 1
1 1
0 0
0 0
1 1
1 1
0 1
0 0
0 0
0 0
1 1
1 1
0 0
1 1
0 0
0 0
0 0
1 1
1 0
1 1
1 1
0 0
1 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 1
0 0
1 1
0 0
0 0
0 0
0 0
1 1
0 0
1 1
0 0
0 1
1 1
1 1
0 0
0 0
0 0
1 1
0 0
0 0
0 1
1 1
1 1
0 0
0 0
1 1
1 1
1 1
1 0
0 0
1 1
1 1
0 0
1 1
1 1
0 0
1 1
1 1
0 0
1 1
0 0
1 0
1 1
0 0
0 0
0 0
0 0
1 1
1 1
1 0
0 0
0 0
1 1
0 0
1 1
0 0
0 0
1 1
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 1
1 1
0 0
0 0
0 0
1 1
1 1
0 0
0 0
1 1
0 0
1 1
0 0
0 0
0 0
0 0
1 0
1 0
1 1
0 0
1 1
0 0
0 0
0 0
1 0
1 1
0 0
0 0
0 0
0 0
1 0
1 1
0 0
1 1
1 1
0 0
1 1
0 0
1 1
0 0
0 0
0 0
0 0
1 0
1 1
1 1
0 0
1 0
0 0
0 0
1 1
1 1
1 1
0 0
0 0
0 0
1 1
1 0
0 0
0 0
1 1
1 1
1 1
0 0
0 0
0 0
0 0
1 0
1 1
1 1
0 0
1 1
0 0
0 0
1 1
0 0
1 1
0 0
0 1
1 1
1 0
1 0
1 1
1 1
